In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re

In [4]:
def intersections(road):
    df = pd.read_csv('_roads3.csv')
    lijst = df['road'].unique()

    matchers = ['N']
    matching = [s for s in lijst if any(xs in s for xs in matchers)]

    df2 = df[df['name'].str.contains('|'.join(matching))].reset_index().drop('index', axis=1)
    side = df2[df2['road'].str.contains('|'.join(matching))].reset_index().drop('index', axis=1)

    side = side.rename(columns={'chainage':'km'})
    side['model_type'] = 'intersection'

    side['intersection'] = ""
    side['length'] = 4
    side = side[side.road == road].reset_index().drop("index", axis=1)
    n = 0

    for i in side.name:
        wordlist = re.sub("[^\w]", " ",  i).split()
        for j in matching:
            if any(j in s for s in wordlist):
                match = [s for s in wordlist if j in s]
                if len(match) == 2:
                    side.loc[n,'intersection'] = str(match[0] + ', ' + match[1])
                elif len(match) == 3:
                    side.loc[n,'intersection'] = str(match[0] + match[1] + match[2])
                elif len(match)==1:
                    side.loc[n,'intersection'] = match[0]
        n += 1

    side = side[['road','km','length','lat','lon','model_type','intersection',]]
        
    return side

In [5]:
def model_structure(road):
    df= pd.read_csv('CandV_Bridges.csv')
    df['model_type'] = 'bridge'
    side = intersections(road)
    df = pd.concat([df,side])
    df_road = df.loc[df.road==road,:].sort_values('km')
    df_road = df_road[['km','length','condition','lat','lon','intersection','model_type']].reset_index().drop('index',axis=1)
    
    df2 = pd.read_csv('_roads3.csv')
    df_start_end = df2.loc[df2.road==road,:]
    df_start_end = df_start_end.iloc[[0, -1]].reset_index().drop('index', axis=1)
    df_start_end = df_start_end.rename(columns={'chainage':'km'})
    df_start_end = df_start_end[['km','lat','lon']]
    df_start_end['id'] = 0
    df_start_end['model_type'] = 0
    df_start_end['length'] = 4
    df_start_end.loc[0,'id'] = 10000
    df_start_end.loc[0,'model_type'] = 'source'
    
    #make this so big that it will definetly end up at the end of the road, will be changed later
    df_start_end.loc[1,'id'] = 100000000
    df_start_end.loc[1,'model_type'] = 'sink'
    
    links = pd.DataFrame(columns=['km','length','condition','lat','lon','intersection'])
    
    for i in range(len(df_road)):
        links = links.append(df_road.iloc[i,:])
        if i == 0:
            links.loc[i,'km'] = df_road.loc[i,'km']
            links.loc[i,'lat'] = (df_road.loc[i,'lat'] + df_start_end.loc[0,'lat'])/2
            links.loc[i,'lon'] = (df_road.loc[i,'lon'] + df_start_end.loc[0,'lon'])/2
        elif i != 0:
            links.loc[i,'km'] = df_road.loc[i,'km'] - df_road.loc[i-1,'km']
            links.loc[i,'lat'] = (df_road.loc[i,'lat'] + df_road.loc[i-1,'lat'])/2
            links.loc[i,'lon'] = (df_road.loc[i,'lon'] + df_road.loc[i-1,'lon'])/2
        if i == len(df_road)-1 and len(df_road) > 1:
            links = links.append(df_start_end.iloc[1,:]).reset_index().drop('index',axis=1)
            links.loc[i+1,'km'] = df_start_end.loc[1,'km'] - df_road.loc[i-1,'km']
            links.loc[i+1,'lat'] = (df_road.loc[i,'lat'] + df_start_end.loc[1,'lat'])/2
            links.loc[i+1,'lon'] = (df_road.loc[i,'lon'] + df_start_end.loc[1,'lon'])/2
        if i == len(df_road)-1 and len(df_road) <= 1:
            links = links.append(df_start_end.iloc[1,:]).reset_index().drop('index',axis=1)
            links.loc[i+1,'km'] = df_start_end.loc[1,'km'] - df_road.loc[0,'km']
            links.loc[i+1,'lat'] = (df_road.loc[i,'lat'] + df_start_end.loc[1,'lat'])/2
            links.loc[i+1,'lon'] = (df_road.loc[i,'lon'] + df_start_end.loc[1,'lon'])/2
    
    links['id'] = (links.index * 2) + 10001
    links['length'] = links['km']*1000
    links['model_type'] = 'link'
    links['condition'] = 'NaN'
    links = links.drop('km',axis=1)
    df_road['id'] = (df_road.index * 2) + 10002
    df_road = pd.concat([df_road,links,df_start_end]).sort_values(['id']).reset_index().drop(['km','index'],axis=1)
    df_road.loc[len(df_road)-1,'id'] = 10000+len(df_road)-1
    df_road['road'] = road
    
    
    return df_road

In [88]:
def combine(road):
    df_road = model_structure(road)
    main_road = intersections(road)
    

    for i in main_road.intersection:
        if i != road:
            a = model_structure(i)
            df_road = pd.concat([df_road, a])

    df_road = df_road.reset_index().drop('index',axis=1)
    df_road['id'] = df_road.index + 10000

    df_road.loc[df_road['model_type'] == 'link', 'intersection'] = 0

    for i in range(len(df_road[df_road.road == road])):
        if df_road.loc[i,'intersection'] in list(main_road.intersection):
            if df_road.loc[i,'road'] == road:
                intersect = df_road.loc[i,'intersection']
                a = df_road[df_road.road == intersect]
                b = a[a.intersection == road]
                c = df_road.loc[i,:]
                if len(b) > 0:
                    q = list(b.index)[0]
                    df_road.loc[q] = c
                    df_road.loc[q,'road'] = intersect
                    df_road.loc[q,'intersection'] = road
                else:
                    n = 10
                    r = df_road[df_road.road == intersect]
                    s = list(r.index)
                    r = r.reset_index()
                    dist = {}
                    for j in range(len(r)):
                        if r.loc[j,'model_type'] == 'bridge':
                            if abs(r.loc[j,'lat'] - df_road.loc[i,'lat'] + r.loc[j,'lon'] - df_road.loc[i,'lon']) < n:
                                n = abs(r.loc[j,'lat'] - df_road.loc[i,'lat'] + r.loc[j,'lon'] - df_road.loc[i,'lon'])
                                m = j
                    print(intersect)
                    print(m)
                    df_road.loc[s[m]] = c
                    df_road.loc[s[m],'road'] = intersect
                    df_road.loc[s[m],'intersection'] = road
    return df_road

In [121]:
def combine2(road1,road2):
    df_road1 = model_structure(road1)
    df_road2 = model_structure(road2)
    main_road1 = intersections(road1)
    main_road2 = intersections(road2)
    

    for i in main_road1.intersection:
        if i != road1:
            a = model_structure(i)
            df_road1 = pd.concat([df_road1, a])
            
    for i in main_road2.intersection:
        if i != road2:
            a = model_structure(i)
            df_road2 = pd.concat([df_road2, a])

    df_road2 = df_road2.reset_index().drop('index', axis=1)
    df_road2 = df_road2.drop(df_road2[df_road2.road == 'N2'].index)

    if road1 in list(main_road2.intersection):
        df_road2 = df_road2.drop(df_road2[df_road2.road == road1].index)
    if road2 in list(main_road1.intersection):
        df_road2 = df_road2.drop(df_road2[df_road2.road == road2].index)
    
    df_road = pd.concat([df_road1,df_road2])
    df_road = df_road.reset_index().drop('index',axis=1)
    df_road['id'] = df_road.index + 10000

    df_road.loc[df_road['model_type'] == 'link', 'intersection'] = 0

    for i in range(len(df_road[df_road.road == road1])):
        if df_road.loc[i,'intersection'] in list(main_road1.intersection):
            if df_road.loc[i,'road'] == road1:
                intersect = df_road.loc[i,'intersection']
                a = df_road[df_road.road == intersect]
                b = a[a.intersection == road1]
                c = df_road.loc[i,:]
                if len(b) > 0:
                    q = list(b.index)[0]
                    df_road.loc[q] = c
                    df_road.loc[q,'road'] = intersect
                    df_road.loc[q,'intersection'] = road1
                else:
                    n = 10
                    r = df_road[df_road.road == intersect]
                    s = list(r.index)
                    r = r.reset_index()
                    dist = {}
                    for j in range(len(r)):
                        if r.loc[j,'model_type'] == 'bridge':
                            if abs(r.loc[j,'lat'] - df_road.loc[i,'lat'] + r.loc[j,'lon'] - df_road.loc[i,'lon']) < n:
                                n = abs(r.loc[j,'lat'] - df_road.loc[i,'lat'] + r.loc[j,'lon'] - df_road.loc[i,'lon'])
                                m = j
                    print(intersect)
                    print(m)
                    df_road.loc[s[m]] = c
                    df_road.loc[s[m],'road'] = intersect
                    df_road.loc[s[m],'intersection'] = road1
                    
    for i in range(len(df_road[df_road.road == road2])):
        s = df_road.index[df_road['road'] == 'N2'].tolist()
        index = s[0]
        if df_road.loc[i+index,'intersection'] in list(main_road2.intersection):
            if df_road.loc[i+index,'road'] == road2:
                intersect = df_road.loc[i+index,'intersection']
                if intersect != road1:
                    a = df_road[df_road.road == intersect]
                    b = a[a.intersection == road2]
                    c = df_road.loc[i+index,:]
                    if len(b) > 0:
                        q = list(b.index)[0]
                        df_road.loc[q] = c
                        df_road.loc[q,'road'] = intersect
                        df_road.loc[q,'intersection'] = road2
                    else:
                        n = 10
                        r = df_road[df_road.road == intersect]
                        s = list(r.index)
                        r = r.reset_index()
                        for j in range(len(r)):
                            if r.loc[j,'model_type'] == 'bridge':
                                if abs(r.loc[j,'lat'] - df_road.loc[i+index,'lat'] + r.loc[j,'lon'] - df_road.loc[i+index,'lon']) < n:
                                    n = abs(r.loc[j,'lat'] - df_road.loc[i+index,'lat'] + r.loc[j,'lon'] - df_road.loc[i+index,'lon'])
                                    m = j
                        print(intersect)
                        print(m)
                        df_road.loc[s[m]] = c
                        df_road.loc[s[m],'road'] = intersect
                        df_road.loc[s[m],'intersection'] = road2
                    
    return df_road

In [125]:
df_road = combine2('N1','N2')
df_road_N1.to_csv('df_road_N1andN2.csv')

N104
6
N102
184
N209
4


In [126]:
df_road

,length,condition,lat,lon,intersection,model_type,id,road
0,4.0,NaN,23.706028,90.443333,NaN,source,10000,N1
1,1800.0,NaN,23.702383,90.451097,0,link,10001,N1
2,11.3,A,23.698739,90.458861,NaN,bridge,10002,N1
3,3125.0,NaN,23.696701,90.473318,0,link,10003,N1
4,6.6,A,23.694664,90.487775,NaN,bridge,10004,N1
...,...,...,...,...,...,...,...,...
3536,4.0,NaN,24.916306,91.949583,N2,intersection,12147,N210
3537,10844.0,NaN,24.882139,91.923152,0,link,13537,N210
3538,4.0,NaN,24.848000,91.896750,N208,intersection,13538,N210
3539,10844.0,NaN,24.848000,91.896750,0,link,13539,N210


In [84]:
s = df_road.index[df_road['road'] == 'N2'].tolist()
s[0]

1303

In [75]:
df_road[df_road.road == 'N2']

,length,condition,lat,lon,intersection,model_type,id,road
1303,4.0,NaN,23.705917,90.521444,NaN,source,11303,N2
1304,0.0,NaN,23.705917,90.521444,0,link,11304,N2
1305,4.0,NaN,23.706083,90.521527,N1,intersection,10006,N2
1306,1511.0,NaN,23.711991,90.518608,0,link,11306,N2
1307,36.5,B,23.718065,90.515771,NaN,bridge,11307,N2
...,...,...,...,...,...,...,...,...
2249,40.0,A,25.171993,92.020065,NaN,bridge,12249,N2
2250,180.0,NaN,25.171521,92.019396,0,link,12250,N2
2251,5.0,A,25.171049,92.018726,NaN,bridge,12251,N2
2252,2056.0,NaN,25.164052,92.018182,0,link,12252,N2


In [89]:
df_road_N2 = combine('N2')
df_road_N2 = df_road_N2.drop(df_road_N2[df_road_N2.road == 'N1'].index)
df_road_N2 = df_road_N2.drop(df_road_N2[df_road_N2.road == 'N2'].index)

N102
184
N209
4


In [43]:
df_all = pd.concat([df_road_N1,df_road_N2]).reset_index().drop('index', axis=1)
df_all

,length,condition,lat,lon,intersection,model_type,id,road
0,4.0,NaN,23.706028,90.443333,NaN,source,10000,N1
1,1800.0,NaN,23.702383,90.451097,0,link,10001,N1
2,11.3,A,23.698739,90.458861,NaN,bridge,10002,N1
3,3125.0,NaN,23.696701,90.473318,0,link,10003,N1
4,6.6,A,23.694664,90.487775,NaN,bridge,10004,N1
...,...,...,...,...,...,...,...,...
3536,4.0,NaN,24.916306,91.949583,N2,intersection,10844,N210
3537,10844.0,NaN,24.882139,91.923152,0,link,12991,N210
3538,4.0,NaN,24.848000,91.896750,N208,intersection,12992,N210
3539,10844.0,NaN,24.848000,91.896750,0,link,12993,N210


In [37]:
df_road_N2[df_road_N2.road == 'N1']

,length,condition,lat,lon,intersection,model_type,id,road
951,4.0,NaN,23.706028,90.443333,NaN,source,10951,N1
952,1800.0,NaN,23.702383,90.451097,0,link,10952,N1
953,11.3,A,23.698739,90.458861,NaN,bridge,10953,N1
954,3125.0,NaN,23.696701,90.473318,0,link,10954,N1
955,6.6,A,23.694664,90.487775,NaN,bridge,10955,N1
...,...,...,...,...,...,...,...,...
2249,1.5,C,20.883070,92.298163,NaN,bridge,12249,N1
2250,247.0,NaN,20.882028,92.297970,0,link,12250,N1
2251,35.9,C,20.880985,92.297777,NaN,bridge,12251,N1
2252,2388.0,NaN,20.871951,92.297930,0,link,12252,N1


In [30]:
df = pd.read_csv('_roads3.csv')
df

,road,chainage,lrp,lat,lon,gap,type,name
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...
1,N1,0.814,LRPSa,23.702917,90.450417,NaN,Culvert,Box Culvert
2,N1,0.822,LRPSb,23.702778,90.450472,NaN,CrossRoad,Intersection with Z1101
3,N1,1.000,LRP001,23.702139,90.451972,NaN,KmPost,Km post missing
4,N1,2.000,LRP002,23.697889,90.460583,NaN,KmPost,Km post missing
...,...,...,...,...,...,...,...,...
51343,Z8943,7.000,LRP007,22.429499,90.785722,NaN,KmPost,Km Post Missing
51344,Z8943,8.000,LRP008,22.430110,90.776333,NaN,KmPost,Km Post Missing
51345,Z8943,8.133,LRP008a,22.430249,90.774888,BS,Bridge,Start of Bridge
51346,Z8943,8.151,LRP008b,22.430249,90.774860,BE,Bridge,End of Bridge
